<a href="https://colab.research.google.com/github/Aida81/Recycle-Item-Detection/blob/main/Recycle_Detection_Tensorflow_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training an Object Detection Model for Raspberry Pi with TensorFlow API using Google COLAB

  This Colab is for training a Tensorflow Model used to detect Recycle Item which is fully trained in Google Colab on Tensorflow 1.13 and then converted to a TFLite model to be used on the Raspberry Pi. 
  
  The Colab here could be easily converted to training any other type of visual detector using the same basic functions but with a different set of labeled images and just changes to the config file and label map files in the training folder.
  
  Therefore, I am training it with my own dataset which 6 classes of Recycle Item consists of cardboard, glass, paper, metal, plastic and trash. 

# Install the version of Tensorflow for this model

Tensorflow GPU v1.13 had been used and this section shows the steps to setup the tensorflow.

In [ ]:
# Uninstall current tensorflow, make sure that you confirm at this step, 
# because it will ask if you are sure
!pip uninstall tensorflow

Install Tensorflow-GPU v1.13

In [ ]:
!pip install tensorflow-gpu==1.13.2

Check python version, should be Python 3.6.9

In [ ]:
!python --version

Python 3.6.9


Verify installed Tensorflow

In [ ]:
import tensorflow as tf
print (tf.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.2


In [ ]:
from google.colab import drive
drive.mount ('/content/gdrive')

In [ ]:
# show current directory
!pwd

/content


Next download the model archive that contains the object detection model example. We need to specifically use the version 1.13 to match the version of Tensorflow that we just installed

In [ ]:
# Get Tensorflow from GitHub and unzip it
!cd /content
!mkdir tensorflow1
!cd /content/tensorflow1
!wget https://github.com/tensorflow/models/archive/r1.13.0.zip
!unzip -q ./r1.13.0.zip
!mv models-r1.13.0/ tensorflow1/models/
!rm r1.13.0.zip

--2021-01-29 04:01:33--  https://github.com/tensorflow/models/archive/r1.13.0.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/tensorflow/models/zip/r1.13.0 [following]
--2021-01-29 04:01:33--  https://codeload.github.com/tensorflow/models/zip/r1.13.0
Resolving codeload.github.com (codeload.github.com)... 140.82.113.10
Connecting to codeload.github.com (codeload.github.com)|140.82.113.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘r1.13.0.zip’

r1.13.0.zip             [<=>                 ] 420.44M  18.8MB/s    in 25s     

2021-01-29 04:01:58 (16.9 MB/s) - ‘r1.13.0.zip’ saved [440860285]



Build and install the Libraries needed to run the example object_detection code

In [ ]:
# Build and Install Tensorflow
!python /content/tensorflow1/models/research/setup.py build
!python /content/tensorflow1/models/research/setup.py install

running build
running install
running bdist_egg
running egg_info
creating object_detection.egg-info
writing object_detection.egg-info/PKG-INFO
writing dependency_links to object_detection.egg-info/dependency_links.txt
writing requirements to object_detection.egg-info/requires.txt
writing top-level names to object_detection.egg-info/top_level.txt
writing manifest file 'object_detection.egg-info/SOURCES.txt'
reading manifest file 'object_detection.egg-info/SOURCES.txt'
writing manifest file 'object_detection.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying object_detection.egg-info/dependency_links.txt -> build/bdist.linux-x86_64/e

Change to the /research folder and build the protos for later use

In [ ]:
# Go into research/ DIR of Tensorflow
%cd /content/tensorflow1/models/research/

# Build protos!
!protoc object_detection/protos/*.proto --python_out=.

/content/tensorflow1/models/research


Set the PYTHONPATH environmental variable

In [ ]:
# Set PYTHONPATH
%set_env PYTHONPATH=/content/tensorflow1/models:/content/tensorflow1/models/research:/content/tensorflow1/models/research/slim:/content/tensorflow1/models:/content/tensorflow1/models/research/object_detection

env: PYTHONPATH=/content/tensorflow1/models:/content/tensorflow1/models/research:/content/tensorflow1/models/research/slim:/content/tensorflow1/models:/content/tensorflow1/models/research/object_detection


Move to the object_detection folder, where you will do most of the remaining work

In [ ]:
# Go into object_detection/ DIR of Tensorflow
%cd object_detection

/content/tensorflow1/models/research/object_detection


Run this if want to know how much time left for current colab session

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.713671231004927


# Model

Download the pretrained Quantized MobileNet V2 Coco model. You will need the quantized model so that it can run using the Google Coral USB modelue for faster calculation on the Raspberry Pi.

In [ ]:
# Now get SSD mobilenet model from GitHub
!wget http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
!tar -xf ./ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
!rm ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz

--2021-01-29 04:02:04--  http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.128, 2607:f8b0:4004:806::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 144806142 (138M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz’

ssd_mobilenet_v2_qu 100%[===================>] 138.10M   137MB/s    in 1.0s    

2021-01-29 04:02:05 (137 MB/s) - ‘ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03.tar.gz’ saved [144806142/144806142]



Download and zip dataset that had been upload in github. The image dataset already annotate by using Classifai and save in .xml file for each image. Then, they also had been splitted into train and test with their own xml file. The image had been converted to tflite file.

In [ ]:
! git clone https://github.com/Aida81/Recycle-Item-Detection.git

Cloning into 'Recycle-Item-Detection'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 33 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (33/33), done.


Change a few line according to the dataset path and class in ssd_mobilenet_v1_quantized_300x300_coco.config

1. Line 9 : num_classes: 6
2. Line 156 : fine_tune_checkpoint: "ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03/model.ckpt" # Path to extracted files of pre-trained model
3.Line 175 :  input_path for training: "training/train.record" # Path to training TFRecord file
4. Line 187 : input_path for evaluation: "training/test.record" # Path to test TFRecord file
5. Line 177 & 191 :  label_map_path: "training/label_map.pbtxt" # Path to label map file

Create folder to hold the test and training images

In [ ]:
!mkdir images

Copy the RecycleItem.zip file into the /images directory, unzip the file and then delete the zip file

In [ ]:
!cp /content/tensorflow1/models/research/object_detection/Recycle-Item-Detection/RecycleItem.zip images/ 
!unzip Recycle-Item-Detection/RecycleItemImage.zip -d images
!rm Recycle-Item-Detection/RecycleItemImage.zip

Archive:  Recycle-Item-Detection/RecycleItemImage.zip
   creating: images/Recycle Item Image/
   creating: images/Recycle Item Image/test/
  inflating: images/Recycle Item Image/test/recycle0004.jpg  
  inflating: images/Recycle Item Image/test/recycle0004_jpg.xml  
  inflating: images/Recycle Item Image/test/recycle0005.jpg  
  inflating: images/Recycle Item Image/test/recycle0005_jpg.xml  
  inflating: images/Recycle Item Image/test/recycle0006.jpg  
  inflating: images/Recycle Item Image/test/recycle0006_jpg.xml  
  inflating: images/Recycle Item Image/test/recycle0007.jpg  
  inflating: images/Recycle Item Image/test/recycle0007_jpg.xml  
  inflating: images/Recycle Item Image/test/recycle0008.jpg  
  inflating: images/Recycle Item Image/test/recycle0008_jpg.xml  
  inflating: images/Recycle Item Image/test/recycle0011.jpg  
  inflating: images/Recycle Item Image/test/recycle0011_jpg.xml  
  inflating: images/Recycle Item Image/test/recycle0012.jpg  
  inflating: images/Recycle Ite

Copy the tensorflow train.py example from the legacy folder into the main object detection_folder. This is the file that will handle the training process.

In [ ]:
#copy the tensoflow train.py exmaple from the legacy folder into the main object detection_folder
!cp /content/tensorflow1/models/research/object_detection/legacy/train.py /content/tensorflow1/models/research/object_detection/

# Time to Train!




Train 

In [ ]:
# verify that the files indicated in the config file are where you say (test.record and train.record especially)
# Run the training!!!
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=/content/tensorflow1/models/research/object_detection/Recycle-Item-Detection/training/ssd_mobilenet_v2_quantized_300x300_coco.config

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

After training is complete you will need to know the number from the latest checkpoint for the next few steps. The number you are looking for looks like this from the files in the training folder: model.ckpt-4696.index



In [ ]:
ls -al training

ls: cannot access 'training': No such file or directory


OPTIONAL: zip the training folder and download it in case you want to restart traing at another time

In [ ]:
### OPTIONAL, if you want to continue training sometimes later ###
# Zip the training folder and download locally, so we can continue in another session if required
!zip -r 'training4696.zip' ./training

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path /content/tensorflow1/models/research/object_detection/training/pipeline.config --trained_checkpoint_prefix training/model.ckpt-4696 --output_directory trained_inference_graph/



# CREATE frozen_inference_graph.pb



A frozen inference graph can be used on a wide variety of machines and after you create one you can use the next steps to test it here in Colab against the test images or you can upload your own to the /test_images folder if you want.  

You will need the number of the latest training checkpoint in this command so be sure you have found it in the preceding steps.  Look for the latest step number in the training folder and replace the number here in the parameters with the step number from your folder:   
--trained_checkpoint_prefix training/model.ckpt-**13144**

In [ ]:
!mkdir inference_graph
# export .pd file for windows
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path=/content/tensorflow1/models/research/object_detection/training/pipeline.config --trained_checkpoint_prefix training/model.ckpt-4696 --output_directory inference_graph

In [ ]:
# frozen_inference_graph.pb
### OPTIONAL ###
# Zip the inference_graph folder, and backup to GDrive, so we can use it for normal predictions on windows
!zip -r 'inference_graph4696.zip' ./inference_graph   VB 

#Use the frozen_inference_graph.pb to visually verify the model works

Move to the /research folder to run the code below and use the frozen inference graph created in the steps above. This .pb file can be used on many different types of machines including linux, mac and windows.

In [ ]:
#!pwd
%cd /content/tensorflow1/models/research


Next set the code to use the frozen_inference_graph.pb created in the steps above and to use the label map used in training the model.

In [ ]:
# run from /content/tensorflow1/models/research

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = 'object_detection/inference_graph/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/tensorflow1/models/research/object_detection/Recycle-Item-Detection/training/labelmap.pbtxt'


print(PATH_TO_FROZEN_GRAPH)
print(PATH_TO_LABELS)

Below is some boiler plate code to setup the Colab to run the inference on local images. 

In [ ]:
# Load a (frozen) Tensorflow model into memory.
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

# Loading label map
# Label maps map indices to category names, so that when our convolution network predicts 5, we know that this corresponds to airplane. Here we use internal utility functions, 
# but anything that returns a dictionary mapping integers to appropriate string labels would be fine
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# helper code
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session(config=config) as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

The next block is the code used to draw the color coded boxes around the item that are identified in the images, you will not need to make any edis here, just run it and move on to the next step.

In [ ]:
import collections
import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements
import matplotlib.pyplot as plt  # pylint: disable=g-import-not-at-top
import numpy as np
import PIL.Image as Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont
import six
import tensorflow as tf
import numpy as np

def draw_bounding_box_on_image_array(image, ymin, xmin, ymax, xmax, color='red', thickness=4, display_str_list=()):
  image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
  draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color, thickness, display_str_list)
  np.copyto(image, np.array(image_pil))

def draw_bounding_box_on_image(image, ymin, xmin, ymax, xmax, color='red', thickness=4, display_str_list=()):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top), (left, top)], width=thickness, fill=color)
  try:
    font = ImageFont.truetype('arial.ttf', 24)
  except IOError:
    font = ImageFont.load_default()

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin), (left + text_width, text_bottom)], fill=color)
    draw.text((left + margin, text_bottom - text_height - margin), display_str, fill='black', font=font)
    text_bottom -= text_height - 2 * margin

def visualize_boxes_and_labels_on_image_array(image, boxes, classes, scores, category_index, instance_masks=None,
    max_boxes_to_draw=20, min_score_thresh=.5, line_thickness=4, groundtruth_box_visualization_color='black'):
  
  # Create a display string (and color) for every box location, group any boxes
  # that correspond to the same location.
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_instance_masks_map = {}
  box_to_instance_boundaries_map = {}
  box_to_keypoints_map = collections.defaultdict(list)

  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]
  for i in range(min(max_boxes_to_draw, boxes.shape[0])):
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      if instance_masks is not None:
        box_to_instance_masks_map[box] = instance_masks[i]
      if scores is None:
        box_to_color_map[box] = groundtruth_box_visualization_color
      else:
        display_str = ''

        if classes[i] in category_index.keys():
          class_name = category_index[classes[i]]['name']
        else:
          class_name = 'N/A'
        display_str = '{}: {}%'.format(display_str, int(100*scores[i]))
        box_to_display_str_map[box].append(display_str)

        box_to_color_map[box] = assigned_color_by_class(classes[i])
        print('scores:', scores[i], ' class: ', category_index[classes[i]]['name'])

  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    draw_bounding_box_on_image_array(image, ymin, xmin, ymax, xmax, color=color, thickness=line_thickness, display_str_list=box_to_display_str_map[box])
  return image

def assigned_color_by_class(class_number):
  if class_number == 1: # cardboard
    color_name = 'Red' # shows as Red
  elif class_number == 2: #glass
    color_name = 'Chartreuse'  #shows as green
  elif
    class_number == 3: #metal
    color_name = 'Blue' #shows blue box
  elif
    class_number == 4: #paper
    color_name == 'Yellow' #shows as yellow
  elif
    class_number == 5: #plastic 
    color_name == 'White' #shows white
  elif
    class_number == 6: #trash
    color_name == 'Orange' #shows orange
  
  return color_name

The following code will load images to have the model run against. If you want to add images to be tested simply place new images in the /test_images folder and add the path to the image into the array below. These images are loaded and placed in one of the steps above.

In [ ]:
IMAGE_SIZE = (12, 8)
TEST_IMAGE_PATHS = [
  '/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0060.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0079.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0088.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0094.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0100.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0103.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0110.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0117.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0135.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0152.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0170.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0190.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0209.jpg'
  ,'/content/tensorflow1/models/research/object_detection/images/RecycleItemImage/test/recycle0225.jpg'
]

print(TEST_IMAGE_PATHS)
!pwd

The last step is to run an inference on each image in the defined array above. The code will first output the image path for each image and then run teh prediction on the image to see if their are any faces and if the face is wearing a mask or not. You can see the score on each prediction in the output. Finally the code will display each image with the boxes and score overlayed and color coded.

In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)

  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)

  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)

  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)

  print(image_path)


  # Visualization of the results of a detection.
  visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      line_thickness=2,
      min_score_thresh=0.30)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)

# Prepare model for Raspberry Pi

First you will need to create the .pb file setup for tflite and then convert that file to a .tflite .

1. CREATE tflite_graph.pb file from your last saved checkpoint.


In [ ]:
%cd /content/tensorflow1/models/research/object_detection/

In [ ]:
# Create TFLite_model folder
!mkdir TFLite_model
#!mkdir inference_graph

Code below will make the tflite_graph.pb file using the latest checkpoint (set trained_checkpoint_prefix based on the latest file in the training folder). Make sure you look in the training folder for the checkpoint with the largest number and replace the "model.ckpt-13144" with the number from your latest checkpoint like this "model.ckpt-{YOUR NUMBER VALUE}". When you have done that then run the code.

The parameter you will need to adjust in the command below looks like this:

--trained_checkpoint_prefix=/content/tensorflow1/models/research/object_detection/training/model.ckpt-13144

but the 13144 number will need to be from the files in your /training directory

In [ ]:
# Freeze the TFLite inference graph for using on a RPI
# Change ????? to your checkpoint of choice (probably the largest one that was created after you stopped the training process)
!python export_tflite_ssd_graph.py --pipeline_config_path=/content/tensorflow1/models/research/object_detection/training/pipeline.config --trained_checkpoint_prefix=/content/tensorflow1/models/research/object_detection/training/model.ckpt-4696 --output_directory=/content/tensorflow1/models/research/object_detection/TFLite_model --add_postprocessing_op=true

2. CONVERT .pb file to TFLite to be used on the Raspberry Pi

This detect.tflite can be used on the RPI since it has been converted to TFLite format and can be then run on the lower powered machines like the Raspberry Pi

In [ ]:
# CONVERT USING TOCO
!rm ./TFLite_model/detect.tflite
!toco --graph_def_file=/content/tensorflow1/models/research/object_detection/TFLite_model/tflite_graph.pb --output_file=/content/tensorflow1/models/research/object_detection/TFLite_model/detect.tflite --input_arrays=normalized_input_image_tensor --output_arrays=TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3 --inference_type=QUANTIZED_UINT8 --mean_values=128 --std_dev_values=128 --change_concat_input_ranges=false --allow_custom_ops --input_shapes=1,300,300,3 --enable_v1_converter

# Saving models and files created 

The best way to have some output of the models you are creating is to zip up the folders and either 

1) download the zip files or

2) use Google Drive to back up the files to somewhere you can download later.

In [ ]:
# Zip the TFLite_Model folder for downloading
%cd /content/tensorflow1/models/research/object_detection

!zip -r 'TFLite_Model081420.zip' TFLite_model

Next use the code below to mount Google Drive to your Colab so that you can easily save files there (much faster than downloading from Colab).

In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!cp  training4696.zip '/content/drive'

In [ ]:
!cp  TFLite_Model '/content/drive'

In [ ]:
!cp  inference_graph.zip '/content/drive/My Drive/machine_learning/SSD_TRAINER'

Then, transfer the .tflite to Raspberry Pi to run on PiCamera